#Applied Natural Language Processing 955G5
##Computer Based Examination, 2023

Remember, you can add cells and change their type (between code and text/markdown) as required to answer the questions.

In [ ]:
# update your candidate number here
candidate_number = 11111111

# Question 1 (50 marks)

This question is about document similarity and information retrieval.

In [4]:
### do not change the code in this cell
# make sure you run this cell
from math import log, sqrt
from nltk.probability import FreqDist
from nltk.corpus import stopwords
import nltk
nltk.download("punkt")
nltk.download("stopwords")
stops = set(stopwords.words('english'))

documents=[["NLTK", ",", "or", "Natural", "Language", "Toolkit" ",", "is", "a", "Python", "package", "that", "you", "can", "use", "for", "NLP", "."],
           ["Neuro-linguistic", "programming", "(", "NLP", ")", "is", "a", "pseudoscientific", "approach", "to", "communication" ",", "personal", "development", "and", "psychotherapy", "."],
           ["Python", "is", "a", "genus", "of", "constricting", "snakes", "in", "the", "Pythonidae", "family", "."]]


query="NLP Python"

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


a) This question deals with the similarity of documents in terms of comparing the words they contain. Describe two approaches to measuring the similarity of individual words. (4 marks)

Cosine Similarity:  For words represented as vectors in a term-frequency space, the cosine similarity is calculated 
using the dot product of the vectors. cos = dot(AB) / |A|*|B|

Jaccard Similarity:  Jaccard similarity measures the similarity between two sets by comparing the intersection of the
 sets to their union. intersection(AB) / union(AB)
 
 

b)
i) Write a function that takes a set of tokenized documents as input and returns the same documents after removing stopwords and punctuation, and applying case normalisation to the remaining tokens. So for example, the documents `[["This", "is", "Alice", "'s", "document", "."], ["Bob", "'s", "document", "."]]` would become `[["alice", "document"], ["bob", "document"]]`. Apply this function to the list `documents` to produce a list called `normalised_documents`. (10 marks)

In [6]:
def normalise(documents):
    normalised_documents = []
    for sentence in documents:
        sent = []
        for w in sentence:
            if w not in stops and w.isalpha():
                sent.append(w.lower())
        normalised_documents.append(sent)
    return normalised_documents

normalised_documents = normalise(documents)
print(normalised_documents)


[['nltk', 'natural', 'language', 'python', 'package', 'use', 'nlp'], ['programming', 'nlp', 'pseudoscientific', 'approach', 'personal', 'development', 'psychotherapy'], ['python', 'genus', 'constricting', 'snakes', 'pythonidae', 'family']]


ii) What impact will stopword removal and case normalisation have on the similarity of documents? (4 marks)

It will make the words about main content in the paper more important, and make the document similarity more 
meaningful and accurate.

iii) Write a function that takes a list of tokenized documents as input and produces a list of bag-of-words representations. That is, each document should be represented as a FreqDist dictionary mapping tokens to the count of each token in the document. For example, the document `["alice", "document"]` would become `FreqDist({'alice': 1, 'document':1})`. Apply this function to the list `normalised_documents` to produce a list called `bags`. (6 marks)

In [7]:
def bag_of_words_representation(normalised_documents):
    bags = []
    for sentence in normalised_documents:
        bag_of_words = FreqDist(sentence)
        bags.append(bag_of_words)
    return bags

bags = bag_of_words_representation(normalised_documents)
print(bags)

[FreqDist({'nltk': 1, 'natural': 1, 'language': 1, 'python': 1, 'package': 1, 'use': 1, 'nlp': 1}), FreqDist({'programming': 1, 'nlp': 1, 'pseudoscientific': 1, 'approach': 1, 'personal': 1, 'development': 1, 'psychotherapy': 1}), FreqDist({'python': 1, 'genus': 1, 'constricting': 1, 'snakes': 1, 'pythonidae': 1, 'family': 1})]


iv) Given the definition below, write a function that takes a list of bag-of-words document representations as input and calculates inverse document frequencies. The output of the function should be dictionary that has tokens as keys and inverse document frequencies as values. 

$$ idf(t) = \log_2 \left( \frac{N}{df(t)} \right) $$

Here, $df(t)$, is the document frequency of the token, $t$, i.e. the number of documents it occurs in, and $N$ is the total number of documents. For example, if alice occurs in 1 out of 2 documents, then the inverse document frequency dictionary should return the value $1 = \log_2(2)$ for the key `'alice'`.

Apply this function to the list `bags` to produce a dictionary called `idf`.

(4 marks)

In [8]:
def calculate_idf(bags):
    N = len(bags)
    idf = {}

    document_frequencies = {}
    for bag in bags:
        for token in bag.keys():
            document_frequencies[token] = document_frequencies.get(token, 0) + 1

    for token, df in document_frequencies.items():
        idf[token] = log((N / df), 2)

    return idf

idf = calculate_idf(bags)
print(idf)

{'nltk': 1.5849625007211563, 'natural': 1.5849625007211563, 'language': 1.5849625007211563, 'python': 0.5849625007211562, 'package': 1.5849625007211563, 'use': 1.5849625007211563, 'nlp': 0.5849625007211562, 'programming': 1.5849625007211563, 'pseudoscientific': 1.5849625007211563, 'approach': 1.5849625007211563, 'personal': 1.5849625007211563, 'development': 1.5849625007211563, 'psychotherapy': 1.5849625007211563, 'genus': 1.5849625007211563, 'constricting': 1.5849625007211563, 'snakes': 1.5849625007211563, 'pythonidae': 1.5849625007211563, 'family': 1.5849625007211563}


v) Write a function that takes a list of bag-of-words document representations and a dictionary of inverse document frequencies and produces a new set of $tfidf$ representations. Apply this function to the list `bags` and the dictionary `idf` to produce a list of document representations called `tfidf_reps`. (4 marks)

In [9]:
def calculate_tfidf(bags, idf):
    tfidf_reps = []

    for bag in bags:
        tfidf_rep = {}
        total_tokens = sum(bag.values())

        for token, tf in bag.items():
            tfidf_rep[token] = tf * idf.get(token, 0) / total_tokens

        tfidf_reps.append(tfidf_rep)

    return tfidf_reps

tfidf_reps = calculate_tfidf(bags, idf)

for tfidf_rep in tfidf_reps:
    print(tfidf_rep)


{'nltk': 0.2264232143887366, 'natural': 0.2264232143887366, 'language': 0.2264232143887366, 'python': 0.08356607153159375, 'package': 0.2264232143887366, 'use': 0.2264232143887366, 'nlp': 0.08356607153159375}
{'programming': 0.2264232143887366, 'nlp': 0.08356607153159375, 'pseudoscientific': 0.2264232143887366, 'approach': 0.2264232143887366, 'personal': 0.2264232143887366, 'development': 0.2264232143887366, 'psychotherapy': 0.2264232143887366}
{'python': 0.0974937501201927, 'genus': 0.26416041678685936, 'constricting': 0.26416041678685936, 'snakes': 0.26416041678685936, 'pythonidae': 0.26416041678685936, 'family': 0.26416041678685936}


c)
i) Write a function that takes a pair of bag-of-word representations and calculates their dot product. (4 marks)

In [10]:
def calculate_dot_product(bag1, bag2):
    
    common_tokens = set(bag1.keys()) & set(bag2.keys())
    dot_product = sum(bag1[token] * bag2[token] for token in common_tokens)
    return dot_product

dot = calculate_dot_product(bags[0], bags[1])
print(dot)

1


ii) Write a function that takes a pair of vectors and calculates their cosine similarity. (4 marks)

In [11]:
def calculate_cosine_similarity(vector1, vector2):
    dot_product = calculate_dot_product(vector1, vector2)

    magnitude1 = sqrt(sum(value**2 for value in vector1.values()))
    magnitude2 = sqrt(sum(value**2 for value in vector2.values()))

    if magnitude1 == 0 or magnitude2 == 0:
        return 0  # To handle the case where one or both vectors have zero magnitude

    cosine_similarity = dot_product / (magnitude1 * magnitude2)
    return cosine_similarity

cosine_similarity_result = calculate_cosine_similarity(bags[0], bags[1])
print(cosine_similarity_result)

0.14285714285714285


iii) Put the string in `query` through the appropriate pre-processing to produce a tfidf bag-of-words representation, and find the index of the most similar document. Print out the corresponding document from the list `documents`. (10 marks)

In [12]:
query_tokens = [w.lower() for w in nltk.word_tokenize(query) if w.lower() not in stops and w.isalpha()]

vocabulary = set()
for sentence in normalised_documents:
    vocabulary.update(sentence)
    
tf_documents = bags
vocabulary.update(query_tokens)

document_count = len(normalised_documents)
idf_values = {term: log(document_count / sum(term in doc for doc in normalised_documents)) for term in vocabulary}

tfidf_documents = []
for tf_doc in tf_documents:
    tfidf_doc = {term: tf * idf_values[term] for term, tf in tf_doc.items()}
    tfidf_documents.append(tfidf_doc)

tfidf_query = {term: query_tokens.count(term) * idf_values[term] for term in vocabulary}

cosine_similarities = [calculate_cosine_similarity(tfidf_query, tfidf_doc) for tfidf_doc in tfidf_documents]
print(cosine_similarities)
# vi) Find the index of the most similar document
most_similar_index = max(range(len(cosine_similarities)), key=cosine_similarities.__getitem__)

print("Most similar document:")
print(" ".join(documents[most_similar_index]))

[0.22731013440410222, 0.1053522533853186, 0.11515227714480396]
Most similar document:
NLTK , or Natural Language Toolkit, is a Python package that you can use for NLP .
